### Initialize

In [ ]:
import src.esh_client as esh
from demo.shared.tools import TENANT_SUFFIX, get_base_url, get_root_path
import requests
import pandas as pd
import os
import sys
from datetime import datetime

sys.path.append(os.path.join(sys.path[0], '..', '..', '..', '..'))


base_url = get_base_url()
root_path = get_root_path()
example_name = sys.path[0].split(os.sep)[-2]


def execute_search_query(query):
    r = requests.post(
        f'{base_url}/v1/query/{example_name}{TENANT_SUFFIX}/latest', json=[query])
    if r.status_code == 200:
        res = r.json()[0]
        if '@odata.count' in res:
            print(f'Total count: {res["@odata.count"]}')
        if res and 'value' in res and res['value']:
            df = pd.DataFrame.from_dict(res['value'])[
                ['firstName', 'lastName', 'age']]
            df.rename(columns={'firstName': 'First Name',
                      'lastName': 'Last Name', 'age': 'Age'}, inplace=True)
            display(df)
        else:
            print('No results')
    else:
        print(f'Error during search. HTTP status code {r.status_code}')
        print(r.json())


### Search for last name "Kennedy"
-> Simple search for a person with the last name. The results show all persons with the last name "Kennedy"

In [ ]:
so = esh.EshObject(
    count=True,
    top=1,
    scope='Person',
    searchQueryFilter=esh.Expression(
        operator=esh.LogicalOperator.AND,
        items=[
            esh.Comparison(
                property=esh.Property(property='lastName'),
                operator=esh.ComparisonOperator.Search,
                value=esh.StringValue(value='Kennedy')
            )
        ]
    )
)
execute_search_query(so.dict(exclude_none=True))


### Search for first name "Alex"
--> Simple search for a person with the first name. The results show all persons with the first name "Alex".

In [ ]:
so = esh.EshObject(
    count=True,
    top=1,
    scope='Person',
    searchQueryFilter=esh.Expression(
        operator=esh.LogicalOperator.AND,
        items=[
            esh.Comparison(
                property=esh.Property(property='firstName'),
                operator=esh.ComparisonOperator.Search,
                value=esh.StringValue(value='Alex'))
        ]))

execute_search_query(so.dict(exclude_none=True))


### Serach for age "37" AND first name "Robert" 
--> Simple search for person with two conditions: Condition 1: Age=37 and Condition 2: First name = Robert. Just in case of the two conditions are true then the results will show something.

In [ ]:
so = esh.EshObject(
    count=True,
    top=1,
    scope='Person',
    searchQueryFilter=esh.Expression(
        operator=esh.LogicalOperator.AND,
        items=[
            esh.Comparison(
                property=esh.Property(property='age'),
                operator=esh.ComparisonOperator.Search,
                value=esh.StringValue(value=37)),
            esh.Comparison(
                property=esh.Property(property='firstName'),
                operator=esh.ComparisonOperator.Search,
                value=esh.StringValue(value='Jerry'))
        ]))
execute_search_query(so.dict(exclude_none=True))


## Search for cases
--> Simple search for cases between date 1.2.2012 and 28.2.2013

In [ ]:
so = esh.EshObject(
    count=True,
    top=1,
    scope='Cases',
    searchQueryFilter=esh.Expression(
        operator=esh.LogicalOperator.AND,
        items=[
            esh.Comparison(
                property=esh.Property(property='caseStartDate'),
                operator=esh.ComparisonOperator.GreaterThanOrEqualCaseInsensitive,
                value=esh.StringValue(value='20120201')),
            esh.Comparison(
                property=esh.Property(property='caseEndDate'),
                operator=esh.ComparisonOperator.LessThanOrEqualCaseInsensitive,
                value=esh.StringValue(value='20130228'))
        ]))

r = requests.post(
    f'{base_url}/v1/query/{example_name}DemoExample/latest', json=[so.dict()])
if r.status_code == 200:
    res = r.json()[0]
    if '@odata.count' in res:
        print(f'Total count: {res["@odata.count"]}')
    if res and 'value' in res and res['value']:

        df = pd.DataFrame.from_dict(res['value'])[
            ['entityDesc', 'entityTime', 'status', 'priority', 'caseStartDate', 'caseEndDate']]
        df.rename(columns={'entityDesc': 'Description',
                  'entityTime': 'Time', 'status': 'Status', 'priority': 'Prio', 'caseStartDate': 'Start Date',
                           'End Date': 'End Date'}, inplace=True)
        display(df)
    else:
        print('No results')
else:
    print(f'Error during search. HTTP status code {r.status_code}')
    print(r.json())


## Search for person who are under 30 years old
--> Simple search with a special condition statement "Lower than". It is searched for persons who are younger than 30 years old. It is also possible to use "Greater than", "Lower Equals", "Greater Equals".

In [ ]:
so = esh.EshObject(
    count=True,
    top=1,
    scope='Person',
    searchQueryFilter=esh.Expression(
        operator=esh.LogicalOperator.AND,
        items=[
            esh.Comparison(

                property=esh.Property(property='age'),
                operator=esh.ComparisonOperator.LessThanOrEqualCaseInsensitive,
                value=esh.StringValue(value=30))
        ]))

execute_search_query(so.dict(exclude_none=True))


## Search for names with wildcard *
--> The wildcard serach is used e.g. to use incomplete words 

In [ ]:
so = esh.EshObject(
    count=True,
    top=3,
    scope='Person',
    searchQueryFilter=esh.Expression(
        operator=esh.LogicalOperator.AND,
        items=[
            esh.Comparison(

                property=esh.Property(property='lastName'),
                operator=esh.ComparisonOperator.Search,
                value=esh.StringValue(value='Kenne*'))
        ]))

execute_search_query(so.dict(exclude_none=True))


## Search for date of birth with ranges between 19750101 - 19801231
--> This simple search shows how ranges can be requested. Here it is searched for a person which is born between 1975 and 1980.

In [ ]:
so = esh.EshObject(
    count=True,
    # top=3,
    scope='Person',
    searchQueryFilter=esh.Expression(
        operator=esh.LogicalOperator.AND,
        items=[
            esh.Comparison(

                property=esh.Property(property='dob'),
                operator=esh.ComparisonOperator.Search,
                value=esh.StringValue(value='[19750101 19801231]'))
        ]))
execute_search_query(so.dict(exclude_none=True))


## Search for persons who are between 30 and 40 and gender=male
--> Simple search for persons with two conditions: It is shown the ranges of the age 30-40 and the gender male

In [ ]:
so = esh.EshObject(
    count=True,
    # top=3,
    scope='Person',
    searchQueryFilter=esh.Expression(
        operator=esh.LogicalOperator.AND,
        items=[
            esh.Comparison(
                property=esh.Property(property='age'),
                operator=esh.ComparisonOperator.Search,
                value=esh.StringValue(value='[30 40]')),
            esh.Comparison(
                property=esh.Property(property='gender'),
                operator=esh.ComparisonOperator.Search,
                value=esh.StringValue(value='male'))
        ]))

execute_search_query(so.dict(exclude_none=True))


## Search for persons where height is between 170 and 180, gender=Male, age between 50-60, country=UK
--> Simple search with several/multiple conditions

In [ ]:
so = esh.EshObject(
    count=True,
    # top=3,
    scope='Person',
    searchQueryFilter=esh.Expression(
        operator=esh.LogicalOperator.AND,
        items=[
            esh.Comparison(
                property=esh.Property(property='height'),
                operator=esh.ComparisonOperator.Search,
                value=esh.StringValue(value='[170 180]')),
            esh.Comparison(
                property=esh.Property(property='gender'),
                operator=esh.ComparisonOperator.Search,
                value=esh.StringValue(value='male')),
            esh.Comparison(
                property=esh.Property(property='age'),
                operator=esh.ComparisonOperator.Search,
                value=esh.StringValue(value='[50 60]')),
            esh.Comparison(
                property=esh.Property(property='country'),
                operator=esh.ComparisonOperator.Search,
                value=esh.StringValue(value='UK'))
        ]))

execute_search_query(so.dict(exclude_none=True))


## Search for person who is a male, ethnicity=AFR, and complexion=Dark

--> Search for persons with multiple conditions

In [ ]:
so = esh.EshObject(
    count=True,
    # top=3,
    scope='Person',
    searchQueryFilter=esh.Expression(
        operator=esh.LogicalOperator.AND,
        items=[
            esh.Comparison(
                property=esh.Property(property='ethnicity'),
                operator=esh.ComparisonOperator.Search,
                value=esh.StringValue(value='AFR')),
            esh.Comparison(
                property=esh.Property(property='gender'),
                operator=esh.ComparisonOperator.Search,
                value=esh.StringValue(value='male')),
            esh.Comparison(
                property=esh.Property(property='complexion'),
                operator=esh.ComparisonOperator.Search,
                value=esh.StringValue(value='Dark'))
        ]))

execute_search_query(so.dict(exclude_none=True))
